In [8]:
from online_attacks.utils.logger import Logger, LoggerParams
from online_attacks.classifiers.mnist import load_mnist_dataset, load_mnist_classifier
from online_attacks.datastream import datastream
from online_attacks.attacks import create_attacker, compute_attack_success_rate
from omegaconf import OmegaConf
import numpy as np
import torch

import sys
sys.path.append("../scripts/")
from online_attacks_sweep import Params

In [27]:
list_records = Logger.list_all_records("/checkpoint/hberard/OnlineAttack/results")
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = load_mnist_dataset(train=False)
for logger in list_records:
    params = logger.load_hparams()
    record = logger.load_record()
    conf = OmegaConf.structured(Params)
    params = OmegaConf.merge(conf, params)
    target_classifier = load_mnist_classifier(params.model_type, index=1, model_dir=params.model_dir, device=device, eval=True)
    source_classifier = load_mnist_classifier(params.model_type, index=params.index, model_dir=params.model_dir, device=device, eval=True)
    attacker = create_attacker(source_classifier, params.attacker_type, params.attacker_params)
    transform = datastream.Compose([datastream.ToDevice(device), datastream.AttackerTransform(attacker),
                                    datastream.ClassifierTransform(target_classifier)])
    fool_rate = []
    for run in record["runs"]:
        permutation = run["permutation"]
        indices = [x[1] for x in run["indices"]]
        target_stream = datastream.BatchDataStream(dataset, batch_size=1000, transform=transform, permutation=permutation)
        stream = target_stream.subset(indices)
        fool_rate.append(compute_attack_success_rate(stream)*100)
    
    mean_fool_rate = np.mean(fool_rate)
    std_fool_rate = np.std(fool_rate)
    print("Fool rate for %s (K=%i): %.2f +/- %.2f"%(params.online_params.online_type.name, params.online_params.K, mean_fool_rate, std_fool_rate))
    
    

Fool rate for OFFLINE (K=100): 100.00 +/- 0.00
Fool rate for STOCHASTIC_VIRTUAL (K=100): 99.39 +/- 0.75
Fool rate for STOCHASTIC_OPTIMISTIC (K=100): 99.60 +/- 0.80
Fool rate for STOCHASTIC_MODIFIED_VIRTUAL (K=100): 99.40 +/- 0.49
Fool rate for RANDOM (K=100): 75.00 +/- 3.46
Fool rate for OFFLINE (K=10): 100.00 +/- 0.00
Fool rate for STOCHASTIC_VIRTUAL (K=10): 100.00 +/- 0.00
Fool rate for STOCHASTIC_OPTIMISTIC (K=10): 100.00 +/- 0.00
Fool rate for STOCHASTIC_MODIFIED_VIRTUAL (K=10): 100.00 +/- 0.00
Fool rate for RANDOM (K=10): 78.00 +/- 19.39
Fool rate for OFFLINE (K=1000): 99.60 +/- 0.00
Fool rate for STOCHASTIC_VIRTUAL (K=1000): 98.92 +/- 0.38
Fool rate for STOCHASTIC_OPTIMISTIC (K=1000): 99.01 +/- 0.36
Fool rate for STOCHASTIC_MODIFIED_VIRTUAL (K=1000): 99.00 +/- 0.29
Fool rate for RANDOM (K=1000): 72.94 +/- 1.43


In [ ]:
fool_rate = []
for i in range(5):
    
mean_fool_rate = np.mean(fool_rate)
std_fool_rate = np.std(fool_rate)
print("Attack success rate (Random): %.4f"%fool_rate)